In [ ]:
WITH InitialSales AS (
    SELECT  
    a.cIDCompany Company_DwID
    ,a.cIDInvoice Invoice_DwID
    ,i.cSalesPoint
    ,a.cSerie Serie 
    ,MC1_GDescription.cDescription Form
    ,a.cIDCustomer Customer_DwID 
    ,a.cIDBranchInvoice BranchDwID
    ,a.cIDTerritory Territory_DwID
    ,a.dEmission Emission
    ,a.xChannelDistribution ChannelDistribution 
    ,a.xReturnReason InvoiceCancelReason
    ,b.cidOrder Order_DwID
    ,a.xInvoiceType InvoiceType 
    ,e.cIDProduct Product_DwID
    ,e.nAmount Amount
    ,e.nUnitValue UnitValue
    ,e.nTotalValue TotalValue
    ,e.MC1LastUpdate
    ,e.nFreegoods Freegoods
    ,g.nSeq Seq
    ,g.nValue Value
    ,g.cIDPricing Pricing_DwID
    ,g.cIDStep Step_DwID ,g.npercentage
    ,f.xIDReasonReturnStock ReasonReturnStock_DwID
    ,f.xIDMotiveDevolution ReasonReturnStock_DwIDDet
    ,h.lCalculated Calculated
    ,h.cIDPricingFormationRule PricingFormationRule_DwID
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_Invoice a
    --################# order invoice #########################
    inner join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderInvoice 
    ) b 
    on a.cIDCompany = b.cIDCompany
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cform
    and a.cSerie = b.cSerie
    and a.cIDBranchInvoice =b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    and b.mc1enabled = 1
    --################## invoice item #####################
    inner join (select *
    FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceItem
    ) e 
    on e.cIDCompany = a.cIDCompany 
    and e.cIDInvoice = a.cIDInvoice 
    and e.cForm = a.cform
    and e.cSerie = a.cSerie
    AND e.cIDBranchInvoice=a.cIDBranchInvoice 
    and e.mc1enabled = 1
    --################ invoice item detail #######################
    inner join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceItemDetail 
    ) g
    on g.cIDCompany = e.cIDCompany
    and g.cIDInvoice = e.cIDInvoice
    and g.cForm = e.cform
    and g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    and g.nSeq = e.nSeq --igualando pasos 
    and g.cidProduct = e.cidProduct --igualando los productos de factura y order
    and g.mc1enabled = 1
    and g.cIDStep in ('00100','00400','00500','00600','00700','00150','00393') --steps para llegar a la venta neta 
    --################### MC1_MC_OrderExt ########################
    left join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderExt
    ) f 
    on f.cIDOrder = b.cIDOrder
    and f.cIDCompany = b.cIDCompany
    and f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
    FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceRequestEmites
    ) m
    on a.cIDCompany = m.cIDCompany
    and a.cIDInvoice = m.cIDInvoice
    and a.cForm = m.cform
    and a.cSerie = m.cSerie
    and a.cIDBranchInvoice =m.cIDBranchInvoice
    and m.mc1enabled = 1
    and UPPER(m.xInvoiceRequestStatus)= 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    inner join (
    select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_PricingFormationRule
    ) h
    on g.cIDStep = h.cIDStep
    and g.cIDPricing = h.cIDPricing
    and g.cIDCompany = h.cIDCompany
    and g.mc1enabled = 1
    and h.mc1enabled = 1
    --and upper(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED' --se repite el paso 00100 y parece aumentar el valor 
    inner join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceExt 
    ) i 
    on a.cIDCompany = i.cIDCompany
    and a.cIDInvoice = i.cIDInvoice
    and a.cForm = i.cform
    and a.cSerie = i.cSerie
    and a.cIDBranchInvoice =i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    left join cat_rtmgb_dv.groglortmba_cz.VW_MC1_GeneralDescription MC1_GDescription
    on a.cForm = MC1_GDescription.cIDDomainType
    and MC1_GDescription.cDomainType = 'cForm'
    where a.xInvoiceType not in ('WTM002','WTM005','ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    and format(a.dEmission , 'yyyy-MM-dd') between '2024-12-19' and '2024-12-21' --informacion de un dia
),
ValidOrders AS (
    SELECT distinct b.cIDOrder, b.cIDProduct,a.cIDBranchinvoice 
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderInvoice a
    inner join cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderItem b 
    on a.cIDOrder = b.cIDOrder 
    and a.cIDCompany = b.cIDCompany 
    and a.cIDBranchInvoice = b.cIDBranchinvoice
    where 1=1
    and a.mc1Enabled = 1
    and b.mc1Enabled = 1
    and format(a.mc1lastupdate , 'yyyy-MM-dd') between '2024-11-19' and '2024-12-21' --informacion de un dia 
),
ValidSales AS (
    SELECT 
        a.* 
    FROM InitialSales a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct AND
        a.BranchDwID = b.cIDBranchInvoice
        AND b.cIDOrder is not null
    --WHERE b.cIDOrder is not null
),
FirstDeduplicates AS(
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.BranchDwID, a.ChannelDistribution, 
            a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.Freegoods, a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType,
            a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID ORDER BY a.mc1LastUpdate DESC, a.ReasonReturnStock_DwID DESC, a.ReasonReturnStock_DwIDDet DESC) as rn
    FROM ValidSales a
    ) sub
    WHERE rn = 1
),
Transformations AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN a.Value ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00100' THEN a.npercentage ELSE 0 END AS precio,
        CASE WHEN a.Step_DwID = '00393' THEN a.Value ELSE 0 END AS desc_impuesto,
        CASE WHEN a.Step_DwID = '00500' THEN a.Value ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00400' THEN a.Value ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN a.Value ELSE 0 END AS Escalafc_ML,
        CASE
            WHEN a.InvoiceType = 'ZBON' AND  a.Step_DwID = '00700'  THEN a.Value  
            WHEN a.Step_DwID = '00600' AND a.Freegoods = 0 THEN a.Value
            WHEN a.Step_DwID = '00350' AND a.Freegoods = 1 THEN a.Value 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '00400' THEN a.npercentage/100 ELSE 0 END AS descuento_porc,
        CASE WHEN a.Step_DwID = '00150' THEN a.npercentage/100 ELSE 0 END AS escalafc_porc,
        CASE WHEN a.Step_DwID = '00500' THEN a.npercentage/100 ELSE 0 END AS promocion_porc,
        CASE
            WHEN a.InvoiceType = 'ZBON' AND  a.Step_DwID = '00700'  THEN a.npercentage/100
            WHEN a.Step_DwID = '00600' AND a.Freegoods = 0 THEN a.npercentage/100
            WHEN a.Step_DwID = '00350' AND a.Freegoods = 1 THEN a.npercentage/100
            ELSE 0
        END AS facdsl_porc
        --CASE WHEN a.Step_DwID = '01200' THEN a.Value ELSE 0 END AS Impuesto1,
        --0 as Impuesto2,
        --CASE WHEN a.Step_DwID = '01000' THEN a.Value ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicates a
),
GroupedSales AS (
    SELECT
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        --SUM(Impuesto1) AS Impuesto1,
        --SUM(Venta_Impuestos) AS Venta_Impuestos,
        --SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML,
        SUM(desc_impuesto) AS desc_impuesto,
        SUM(precio) AS precio,
        SUM(descuento_porc) AS descuento_porc,
        SUM(escalafc_porc) AS escalafc_porc,
        SUM(promocion_porc) AS promocion_porc,
        SUM(facdsl_porc) AS facdsl_porc
    FROM Transformations
    GROUP BY
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue
),
GroupesSales2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Escalafc_ML) AS Escalafc_ML,
        SUM(desc_impuesto) AS desc_impuesto,
        SUM(precio) AS precio,
        SUM(descuento_porc) AS descuento_porc,
        SUM(escalafc_porc) AS escalafc_porc,
        SUM(promocion_porc) AS promocion_porc,
        SUM(facdsl_porc) AS facdsl_porc
    FROM GroupedSales
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Serie,
        Territory_DwID,
        UnitValue
),
TransformationFACDSL AS(
    SELECT
        a.BranchDwID,
        a.ChannelDistribution,
        a.Company_DwID,
        a.cSalesPoint,
        a.Customer_DwID,
        a.Emission,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType,
        a.Order_DwID,
        a.Product_DwID,
        a.ReasonReturnStock_DwID,
        a.ReasonReturnStock_DwIDDet,
        a.Serie,
        a.Territory_DwID,
        a.Valor_Bruto,
        a.Amount,
        --a.desc_impuesto,
        --a.precio,
        --a.descuento_porc,
        --a.escalafc_porc,
        --a.promocion_porc,
        --a.facdsl_porc,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.facdsl_porc ELSE a.Facdsl_ML END as Facdsl_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.promocion_porc ELSE a.Promocion_ML END as Promocion_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.descuento_porc ELSE a.Descuento_ML END as Descuento_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.escalafc_porc ELSE a.Escalafc_ML END as Escalafc_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.precio ELSE a.UnitValue END as UnitValue
    FROM GroupesSales2 a
),
InvoiceItemDetailJoin AS (
    SELECT a.cIDCompany, a.cIDBranchInvoice, a.cSerie, i.cSalesPoint, a.cIDInvoice, 
        MC1_GDescription.cDescription cForm, a.cIDProduct, 
        FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm') MC1LastUpdate, c.nfreegoods, 
        a.nValue Value2, a.cidstep Step_DwID, a.npercentage, b.xInvoiceType InvoiceType
    FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceItemDetail a
    INNER JOIN (
        SELECT * 
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_Invoice 
        WHERE MC1Enabled = 1
        AND FORMAT(DEmission, 'yyyy-MM-dd') BETWEEN '2024-12-19' AND '2024-12-21' 
        AND xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR')
    ) b ON a.cIDBranchInvoice = b.cIDBranchInvoice
    AND a.cSerie = b.cSerie
    AND a.cIDInvoice = b.cIDInvoice
    AND a.cForm = b.cForm
    INNER JOIN (
        SELECT * 
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_Invoiceitem
        WHERE MC1Enabled = 1
    ) c ON a.cIDBranchInvoice = c.cIDBranchInvoice
    AND a.cSerie = c.cSerie
    AND a.cIDInvoice = c.cIDInvoice
    AND a.cForm = c.cForm
    AND a.cIDProduct = c.cIDProduct
    AND a.nSeq = c.nSeq
    INNER JOIN (
        SELECT * 
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceExt
    ) i ON a.cIDCompany = i.cIDCompany
    AND a.cIDInvoice = i.cIDInvoice
    AND a.cForm = i.cform
    AND a.cSerie = i.cSerie
    AND a.cIDBranchInvoice = i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    LEFT JOIN GRO_BA_RTM.VW_MC1_GeneralDescription MC1_GDescription
    ON a.cForm = MC1_GDescription.cIDDomainType
    AND MC1_GDescription.cDomainType = 'cForm'
    WHERE 1=1
    AND a.cIDStep IN ('00100', '00400', '00500', '00600', '00700', '00150', '00393', '00395')
    AND a.MC1Enabled = 1
    GROUP BY a.cIDCompany, a.cIDBranchInvoice, a.cSerie, i.cSalesPoint, a.cIDInvoice, 
        a.cIDProduct, FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm'), c.nfreegoods, 
        a.nValue, a.cidstep, a.npercentage, b.xInvoiceType, MC1_GDescription.cDescription
    --ORDER BY a.MC1LastUpdate DESC
),
InvoiceDetailTransformations AS (
    SELECT 
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN a.Value2 ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00100' THEN a.npercentage ELSE 0 END AS precio,
        CASE WHEN a.Step_DwID = '00393' THEN a.Value2 ELSE 0 END AS desc_impuesto,
        CASE WHEN a.Step_DwID = '00395' THEN a.Value2 ELSE 0 END AS desc_iva,
        CASE WHEN a.Step_DwID = '00700' THEN a.Value2 ELSE 0 END AS valor_neto,
        CASE WHEN a.Step_DwID = '00400' THEN a.npercentage/100 ELSE 0 END AS descuento_porc,
        CASE WHEN a.Step_DwID = '00150' THEN a.npercentage/100 ELSE 0 END AS escalafc_porc,
        CASE WHEN a.Step_DwID = '00500' THEN a.npercentage/100 ELSE 0 END AS promocion_porc,
        CASE
            WHEN a.InvoiceType = 'ZBON' AND  a.Step_DwID = '00700'  THEN a.npercentage/100
            WHEN a.Step_DwID = '00600' AND a.nfreegoods = 0 THEN a.npercentage/100
            WHEN a.Step_DwID = '00350' AND a.nfreegoods = 1 THEN a.npercentage/100
            ELSE 0
        END AS facdsl_porc
    FROM InvoiceItemDetailJoin a
),
InvoiceItemDetailDeduplicated AS (
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.cForm, a.cIDBranchInvoice, a.cIDCompany, a.cIDInvoice, a.cIDProduct, a.cSalesPoint, a.cSerie, a.nfreegoods, a.Step_DwID
        ORDER BY a.MC1LastUpdate DESC) as rn
    FROM InvoiceDetailTransformations a
    ) sub
    WHERE rn = 1
),
InvoiceDetailGenerated AS (
    SELECT 
        cIDCompany Company_DwID,
        cIDBranchInvoice BranchInvoice_DwID, 
        cSerie Serie, 
        cSalesPoint,
        cIDInvoice Invoice_DwID, 
        cForm Form, 
        cIDProduct Product_DwID, 
        MC1LastUpdate, 
        nfreegoods FreeGoods, 
        Step_DwID, 
        npercentage, 
        InvoiceType,
        Valor_Bruto,
        precio,
        desc_impuesto,
        desc_iva,
        descuento_porc,
        escalafc_porc,
        promocion_porc,
        facdsl_porc,
        CASE WHEN nfreegoods = 1 THEN 0 ELSE Value2 END AS Value2,
        CASE WHEN nfreegoods = 1 THEN 0 ELSE valor_neto END AS valor_neto
    FROM InvoiceItemDetailDeduplicated
),
InvoiceDetailGrouped AS (
    SELECT
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie,
        SUM(Value2) AS facdet, 
        SUM(valor_neto) AS valor_neto,
        SUM(descuento_porc) AS descuento_porc,
        SUM(escalafc_porc) AS escalafc_porc, 
        SUM(promocion_porc) AS promocion_porc, 
        SUM(facdsl_porc) AS facdsl_porc, 
        SUM(Valor_Bruto) AS Valor_Bruto, 
        SUM(desc_impuesto) AS desc_impuesto, 
        SUM(desc_iva) AS desc_iva
    FROM InvoiceDetailGenerated
    GROUP BY 
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie
),
InvoiceDetailLastTransformations AS (
    SELECT 
        a.Company_DwID,
        a.BranchInvoice_DwID,
        a.Form,
        a.Invoice_DwID,
        a.Product_DwID,
        a.Serie, 
        a.valor_neto,
        a.descuento_porc,
        a.escalafc_porc, 
        a.promocion_porc, 
        a.facdsl_porc, 
        a.Valor_Bruto, 
        a.desc_impuesto, 
        a.desc_iva,
        CASE WHEN a.desc_impuesto <> 0 
        THEN a.desc_iva - a.desc_impuesto - a.Valor_Bruto * a.descuento_porc - a.Valor_Bruto * a.escalafc_porc - a.Valor_Bruto * a.facdsl_porc - a.Valor_Bruto * a.promocion_porc
        ELSE a.valor_neto
        END AS facdet
    FROM InvoiceDetailGrouped a
),
FinalJoin AS (
    SELECT 
        a.*,
        b.facdet
    FROM TransformationFACDSL a
    INNER JOIN InvoiceDetailLastTransformations b ON
        a.Company_DwID = b.Company_DwID AND
        a.BranchDwID = b.BranchInvoice_DwID AND
        a.Product_DwID = b.Product_DwID AND
        a.Serie = b.Serie AND
        a.Form = b.Form AND
        a.Invoice_DwID = b.Invoice_DwID
),
AddColumns AS (
    SELECT 
        a.BranchDwID codigo_agencia,
        a.ChannelDistribution,
        CASE WHEN a.Company_DwID='AR' THEN 'ARGENTINA' ELSE 'REV' END as pais,
        a.cSalesPoint,
        a.Customer_DwID codigo_cliente,
        CONVERT(VARCHAR(10), CAST(a.Emission AS DATE), 120) fecha,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType AG19ID,
        a.Order_DwID,
        a.Product_DwID codigo_producto,
        a.ReasonReturnStock_DwID,
        a.ReasonReturnStock_DwIDDet,
        a.Serie,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') as codigo_ruta,
        a.Valor_Bruto,
        a.Facdsl_ML,
        a.Promocion_ML,
        a.Descuento_ML,
        a.Escalafc_ML,
        a.UnitValue,
        a.Amount,
        CONCAT(
            RIGHT(CONCAT(REPLICATE('0', 4), a.Form), 4), 
            a.Serie, 
            RIGHT(a.cSalesPoint, 4), 
            a.Invoice_DwID
        ) as numero,
        a.ChannelDistribution as cod_linea_ruta,
        'FALSE' as FACCON,
        0 as FACDSL,
        null as dif_precio,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN 0 - facdet
            ELSE facdet
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN a.ReasonReturnStock_DwID
            WHEN NOT (a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX') THEN a.InvoiceCancelReason
            ELSE 'REVISAR'
        END AS Id_Motivo
    FROM FinalJoin a
),
Subchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, a.cIDCustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN cat_rtmgb_dv.groglortmba_cz.VW_MC1_CustomerDivisionSector b 
        ON a.cidcustomer = b.cidcustomer
        WHERE b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
JoinWithSubchannel AS (
    SELECT
        a.*,
        b.xSubChannelDistribution
    FROM AddColumns a
    LEFT JOIN  Subchannel b
    ON a.codigo_cliente = b.cIDCustomer
    AND a.codigo_agencia = b.cidbranchinvoice
),
ModifyColumns AS (
    SELECT 
        a.codigo_agencia,
        a.codigo_cliente CLICOD,
        CONVERT(VARCHAR(10), a.fecha, 120) Fecha_vta,
        a.Invoice_DwID no_factura,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.ReasonReturnStock_DwIDDet,
        a.codigo_ruta,
        a.UnitValue precio,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Facdsl_ML
            ELSE a.Facdsl_ML END AS FACDSL$ ,
         CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Promocion_ML
            ELSE a.Promocion_ML END AS promocion,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Descuento_ML
            ELSE a.Descuento_ML END AS descuento,
        a.Amount cantidad,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZBON' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Escalafc_ML
            ELSE a.Escalafc_ML END AS Escalafc_ML,
        a.numero,
        a.dif_precio,
        a.FACCON,
        a.FACDSL,
        a.facdet FACDET,
        CASE  
    		WHEN a.cod_linea_ruta <> a.xSubChannelDistribution AND a.xSubChannelDistribution IS NOT NULL THEN a.xSubChannelDistribution ELSE a.cod_linea_ruta 
		END AS cod_linea_ruta,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            WHEN upper(a.Id_Motivo)<>'WTM001' THEN a.Id_Motivo
            ELSE a.ReasonReturnStock_DwIDDet END as id_motivoventa,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            WHEN upper(a.Id_Motivo)='WTM002' and a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason 
            ELSE a.Id_Motivo END as Id_Motivo
    FROM JoinWithSubchannel a 
),
ReturnReason AS (
    SELECT 
        CASE
            WHEN cDescription IS NOT NULL AND cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_GeneralDescription
    Where cDomainType = 'xReturnReason'
    AND lEnabled = 1
    AND UPPER(cDescription) <> 'SOBRESTOCK EN TIENDA'
),
Motivos AS(
    SELECT 
        CASE
            WHEN cDescription IS NOT NULL AND cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_GeneralDescription
    Where cDomainType in ('xIDReasonReturnStock', 'xIDMotiveDevolution')
    and cIDLanguage = '2'
    AND lEnabled = 1
    AND UPPER(cDescription) <> 'SOBRESTOCK EN TIENDA'
),
FinalMC1Argentina as (
    SELECT 
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.no_factura,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.FACDSL$,
        a.promocion,
        a.descuento,
        a.cantidad,
        a.Escalafc_ML,
        a.numero,
        a.dif_precio,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.cod_linea_ruta,
        a.id_motivoventa,
        a.Id_Motivo,
        CASE
            WHEN b.Description is null
            THEN b.Description
            ELSE CONCAT(b.Description, '-', c.Description)
        END AS factura_cliente
    FROM ModifyColumns a
    LEFT JOIN ReturnReason b
    ON a.InvoiceCancelReason = b.cIDDomainType
    LEFT JOIN Motivos c
    ON a.id_motivoventa = c.cIDDomainType
    Where a.codigo_ruta NOT LIKE '%0A%'
)
SELECT * FROM FinalMC1Argentina

In [ ]:
WITH InitialSales AS (
    SELECT  
    a.cIDCompany Company_DwID
    ,a.cIDInvoice Invoice_DwID
    ,i.cSalesPoint
    ,a.cSerie Serie 
    ,MC1_GDescription.cDescription Form
    ,a.cIDCustomer Customer_DwID 
    ,a.cIDBranchInvoice BranchDwID
    ,a.cIDTerritory Territory_DwID
    ,a.dEmission Emission
    ,a.xChannelDistribution ChannelDistribution 
    ,a.xReturnReason InvoiceCancelReason
    ,b.cidOrder Order_DwID
    ,a.xInvoiceType InvoiceType 
    ,e.cIDProduct Product_DwID
    ,e.nAmount Amount
    ,e.nUnitValue UnitValue
    ,e.nTotalValue TotalValue
    ,e.MC1LastUpdate
    ,e.nFreegoods Freegoods
    ,g.nSeq Seq
    ,g.nValue Value
    ,g.cIDPricing Pricing_DwID
    ,g.cIDStep Step_DwID ,g.npercentage
    ,f.xIDReasonReturnStock ReasonReturnStock_DwID
    ,f.xIDMotiveDevolution ReasonReturnStock_DwIDDet
    ,h.lCalculated Calculated
    ,h.cIDPricingFormationRule PricingFormationRule_DwID
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_Invoice a
    --################# order invoice #########################
    inner join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderInvoice 
    ) b 
    on a.cIDCompany = b.cIDCompany
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cform
    and a.cSerie = b.cSerie
    and a.cIDBranchInvoice =b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    and b.mc1enabled = 1
    --################## invoice item #####################
    inner join (select *
    FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceItem
    ) e 
    on e.cIDCompany = a.cIDCompany 
    and e.cIDInvoice = a.cIDInvoice 
    and e.cForm = a.cform
    and e.cSerie = a.cSerie
    AND e.cIDBranchInvoice=a.cIDBranchInvoice 
    and e.mc1enabled = 1
    --################ invoice item detail #######################
    inner join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceItemDetail 
    ) g
    on g.cIDCompany = e.cIDCompany
    and g.cIDInvoice = e.cIDInvoice
    and g.cForm = e.cform
    and g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    and g.nSeq = e.nSeq --igualando pasos 
    and g.cidProduct = e.cidProduct --igualando los productos de factura y order
    and g.mc1enabled = 1
    and g.cIDStep in ('00100','00400','00500','00600','00700','00150','00393') --steps para llegar a la venta neta 
    --################### MC1_MC_OrderExt ########################
    left join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderExt
    ) f 
    on f.cIDOrder = b.cIDOrder
    and f.cIDCompany = b.cIDCompany
    and f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
    FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceRequestEmites
    ) m
    on a.cIDCompany = m.cIDCompany
    and a.cIDInvoice = m.cIDInvoice
    and a.cForm = m.cform
    and a.cSerie = m.cSerie
    and a.cIDBranchInvoice =m.cIDBranchInvoice
    and m.mc1enabled = 1
    and UPPER(m.xInvoiceRequestStatus)= 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    inner join (
    select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_PricingFormationRule
    ) h
    on g.cIDStep = h.cIDStep
    and g.cIDPricing = h.cIDPricing
    and g.cIDCompany = h.cIDCompany
    and g.mc1enabled = 1
    and h.mc1enabled = 1
    --and upper(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED' --se repite el paso 00100 y parece aumentar el valor 
    inner join (select *
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceExt 
    ) i 
    on a.cIDCompany = i.cIDCompany
    and a.cIDInvoice = i.cIDInvoice
    and a.cForm = i.cform
    and a.cSerie = i.cSerie
    and a.cIDBranchInvoice =i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    left join cat_rtmgb_dv.groglortmba_cz.VW_MC1_GeneralDescription MC1_GDescription
    on a.cForm = MC1_GDescription.cIDDomainType
    and MC1_GDescription.cDomainType = 'cForm'
    where a.xInvoiceType not in ('WTM002','WTM005','ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    and format(a.dEmission , 'yyyy-MM-dd') between '2024-12-19' and '2024-12-21' --informacion de un dia
),
ValidOrders AS (
    SELECT distinct b.cIDOrder, b.cIDProduct,a.cIDBranchinvoice 
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderInvoice a
    inner join cat_rtmgb_dv.groglortmba_cz.VW_MC1_OrderItem b 
    on a.cIDOrder = b.cIDOrder 
    and a.cIDCompany = b.cIDCompany 
    and a.cIDBranchInvoice = b.cIDBranchinvoice
    where 1=1
    and a.mc1Enabled = 1
    and b.mc1Enabled = 1
    and format(a.mc1lastupdate , 'yyyy-MM-dd') between '2024-11-19' and '2024-12-21' --informacion de un dia 
),
ValidSales AS (
    SELECT 
        a.* 
    FROM InitialSales a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct AND
        a.BranchDwID = b.cIDBranchInvoice
        AND b.cIDOrder is not null
    --WHERE b.cIDOrder is not null
),
FirstDeduplicates AS(
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.BranchDwID, a.ChannelDistribution, 
            a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.Freegoods, a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType,
            a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID ORDER BY a.mc1LastUpdate DESC, a.ReasonReturnStock_DwID DESC, a.ReasonReturnStock_DwIDDet DESC) as rn
    FROM ValidSales a
    ) sub
    WHERE rn = 1
),
Transformations AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN a.Value ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00100' THEN a.npercentage ELSE 0 END AS precio,
        CASE WHEN a.Step_DwID = '00393' THEN a.Value ELSE 0 END AS desc_impuesto,
        CASE WHEN a.Step_DwID = '00500' THEN a.Value ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00400' THEN a.Value ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN a.Value ELSE 0 END AS Escalafc_ML,
        CASE
            WHEN a.InvoiceType = 'ZBON' AND  a.Step_DwID = '00700'  THEN a.Value  
            WHEN a.Step_DwID = '00600' AND a.Freegoods = 0 THEN a.Value
            WHEN a.Step_DwID = '00350' AND a.Freegoods = 1 THEN a.Value 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '00400' THEN a.npercentage/100 ELSE 0 END AS descuento_porc,
        CASE WHEN a.Step_DwID = '00150' THEN a.npercentage/100 ELSE 0 END AS escalafc_porc,
        CASE WHEN a.Step_DwID = '00500' THEN a.npercentage/100 ELSE 0 END AS promocion_porc,
        CASE
            WHEN a.InvoiceType = 'ZBON' AND  a.Step_DwID = '00700'  THEN a.npercentage/100
            WHEN a.Step_DwID = '00600' AND a.Freegoods = 0 THEN a.npercentage/100
            WHEN a.Step_DwID = '00350' AND a.Freegoods = 1 THEN a.npercentage/100
            ELSE 0
        END AS facdsl_porc
        --CASE WHEN a.Step_DwID = '01200' THEN a.Value ELSE 0 END AS Impuesto1,
        --0 as Impuesto2,
        --CASE WHEN a.Step_DwID = '01000' THEN a.Value ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicates a
),
GroupedSales AS (
    SELECT
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        --SUM(Impuesto1) AS Impuesto1,
        --SUM(Venta_Impuestos) AS Venta_Impuestos,
        --SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML,
        SUM(desc_impuesto) AS desc_impuesto,
        SUM(precio) AS precio,
        SUM(descuento_porc) AS descuento_porc,
        SUM(escalafc_porc) AS escalafc_porc,
        SUM(promocion_porc) AS promocion_porc,
        SUM(facdsl_porc) AS facdsl_porc
    FROM Transformations
    GROUP BY
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue
),
GroupesSales2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Escalafc_ML) AS Escalafc_ML,
        SUM(desc_impuesto) AS desc_impuesto,
        SUM(precio) AS precio,
        SUM(descuento_porc) AS descuento_porc,
        SUM(escalafc_porc) AS escalafc_porc,
        SUM(promocion_porc) AS promocion_porc,
        SUM(facdsl_porc) AS facdsl_porc
    FROM GroupedSales
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        ReasonReturnStock_DwIDDet,
        Serie,
        Territory_DwID,
        UnitValue
),
TransformationFACDSL AS(
    SELECT
        a.BranchDwID,
        a.ChannelDistribution,
        a.Company_DwID,
        a.cSalesPoint,
        a.Customer_DwID,
        a.Emission,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType,
        a.Order_DwID,
        a.Product_DwID,
        a.ReasonReturnStock_DwID,
        a.ReasonReturnStock_DwIDDet,
        a.Serie,
        a.Territory_DwID,
        a.Valor_Bruto,
        a.Amount,
        --a.desc_impuesto,
        --a.precio,
        --a.descuento_porc,
        --a.escalafc_porc,
        --a.promocion_porc,
        --a.facdsl_porc,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.facdsl_porc ELSE a.Facdsl_ML END as Facdsl_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.promocion_porc ELSE a.Promocion_ML END as Promocion_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.descuento_porc ELSE a.Descuento_ML END as Descuento_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.Valor_Bruto * a.escalafc_porc ELSE a.Escalafc_ML END as Escalafc_ML,
       CASE WHEN a.desc_impuesto <> 0 THEN a.precio ELSE a.UnitValue END as UnitValue
    FROM GroupesSales2 a
),
InvoiceItemDetailJoin AS (
    SELECT a.cIDCompany, a.cIDBranchInvoice, a.cSerie, i.cSalesPoint, a.cIDInvoice, 
        MC1_GDescription.cDescription cForm, a.cIDProduct, 
        FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm') MC1LastUpdate, c.nfreegoods, 
        a.nValue Value2, a.cidstep Step_DwID, a.npercentage, b.xInvoiceType InvoiceType
    FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceItemDetail a
    INNER JOIN (
        SELECT * 
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_Invoice 
        WHERE MC1Enabled = 1
        AND FORMAT(DEmission, 'yyyy-MM-dd') BETWEEN '2024-12-19' AND '2024-12-21' 
        AND xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR')
    ) b ON a.cIDBranchInvoice = b.cIDBranchInvoice
    AND a.cSerie = b.cSerie
    AND a.cIDInvoice = b.cIDInvoice
    AND a.cForm = b.cForm
    INNER JOIN (
        SELECT * 
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_Invoiceitem
        WHERE MC1Enabled = 1
    ) c ON a.cIDBranchInvoice = c.cIDBranchInvoice
    AND a.cSerie = c.cSerie
    AND a.cIDInvoice = c.cIDInvoice
    AND a.cForm = c.cForm
    AND a.cIDProduct = c.cIDProduct
    AND a.nSeq = c.nSeq
    INNER JOIN (
        SELECT * 
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_InvoiceExt
    ) i ON a.cIDCompany = i.cIDCompany
    AND a.cIDInvoice = i.cIDInvoice
    AND a.cForm = i.cform
    AND a.cSerie = i.cSerie
    AND a.cIDBranchInvoice = i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    LEFT JOIN GRO_BA_RTM.VW_MC1_GeneralDescription MC1_GDescription
    ON a.cForm = MC1_GDescription.cIDDomainType
    AND MC1_GDescription.cDomainType = 'cForm'
    WHERE 1=1
    AND a.cIDStep IN ('00100', '00400', '00500', '00600', '00700', '00150', '00393', '00395')
    AND a.MC1Enabled = 1
    GROUP BY a.cIDCompany, a.cIDBranchInvoice, a.cSerie, i.cSalesPoint, a.cIDInvoice, 
        a.cIDProduct, FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm'), c.nfreegoods, 
        a.nValue, a.cidstep, a.npercentage, b.xInvoiceType, MC1_GDescription.cDescription
    --ORDER BY a.MC1LastUpdate DESC
),
InvoiceDetailTransformations AS (
    SELECT 
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN a.Value2 ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00100' THEN a.npercentage ELSE 0 END AS precio,
        CASE WHEN a.Step_DwID = '00393' THEN a.Value2 ELSE 0 END AS desc_impuesto,
        CASE WHEN a.Step_DwID = '00395' THEN a.Value2 ELSE 0 END AS desc_iva,
        CASE WHEN a.Step_DwID = '00700' THEN a.Value2 ELSE 0 END AS valor_neto,
        CASE WHEN a.Step_DwID = '00400' THEN a.npercentage/100 ELSE 0 END AS descuento_porc,
        CASE WHEN a.Step_DwID = '00150' THEN a.npercentage/100 ELSE 0 END AS escalafc_porc,
        CASE WHEN a.Step_DwID = '00500' THEN a.npercentage/100 ELSE 0 END AS promocion_porc,
        CASE
            WHEN a.InvoiceType = 'ZBON' AND  a.Step_DwID = '00700'  THEN a.npercentage/100
            WHEN a.Step_DwID = '00600' AND a.nfreegoods = 0 THEN a.npercentage/100
            WHEN a.Step_DwID = '00350' AND a.nfreegoods = 1 THEN a.npercentage/100
            ELSE 0
        END AS facdsl_porc
    FROM InvoiceItemDetailJoin a
),
InvoiceItemDetailDeduplicated AS (
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.cForm, a.cIDBranchInvoice, a.cIDCompany, a.cIDInvoice, a.cIDProduct, a.cSalesPoint, a.cSerie, a.nfreegoods, a.Step_DwID
        ORDER BY a.MC1LastUpdate DESC) as rn
    FROM InvoiceDetailTransformations a
    ) sub
    WHERE rn = 1
),
InvoiceDetailGenerated AS (
    SELECT 
        cIDCompany Company_DwID,
        cIDBranchInvoice BranchInvoice_DwID, 
        cSerie Serie, 
        cSalesPoint,
        cIDInvoice Invoice_DwID, 
        cForm Form, 
        cIDProduct Product_DwID, 
        MC1LastUpdate, 
        nfreegoods FreeGoods, 
        Step_DwID, 
        npercentage, 
        InvoiceType,
        Valor_Bruto,
        precio,
        desc_impuesto,
        desc_iva,
        descuento_porc,
        escalafc_porc,
        promocion_porc,
        facdsl_porc,
        CASE WHEN nfreegoods = 1 THEN 0 ELSE Value2 END AS Value2,
        CASE WHEN nfreegoods = 1 THEN 0 ELSE valor_neto END AS valor_neto
    FROM InvoiceItemDetailDeduplicated
),
InvoiceDetailGrouped AS (
    SELECT
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie,
        SUM(Value2) AS facdet, 
        SUM(valor_neto) AS valor_neto,
        SUM(descuento_porc) AS descuento_porc,
        SUM(escalafc_porc) AS escalafc_porc, 
        SUM(promocion_porc) AS promocion_porc, 
        SUM(facdsl_porc) AS facdsl_porc, 
        SUM(Valor_Bruto) AS Valor_Bruto, 
        SUM(desc_impuesto) AS desc_impuesto, 
        SUM(desc_iva) AS desc_iva
    FROM InvoiceDetailGenerated
    GROUP BY 
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie
),
InvoiceDetailLastTransformations AS (
    SELECT 
        a.Company_DwID,
        a.BranchInvoice_DwID,
        a.Form,
        a.Invoice_DwID,
        a.Product_DwID,
        a.Serie, 
        a.valor_neto,
        a.descuento_porc,
        a.escalafc_porc, 
        a.promocion_porc, 
        a.facdsl_porc, 
        a.Valor_Bruto, 
        a.desc_impuesto, 
        a.desc_iva,
        CASE WHEN a.desc_impuesto <> 0 
        THEN a.desc_iva - a.desc_impuesto - a.Valor_Bruto * a.descuento_porc - a.Valor_Bruto * a.escalafc_porc - a.Valor_Bruto * a.facdsl_porc - a.Valor_Bruto * a.promocion_porc
        ELSE a.valor_neto
        END AS facdet
    FROM InvoiceDetailGrouped a
),
FinalJoin AS (
    SELECT 
        a.*,
        b.facdet
    FROM TransformationFACDSL a
    INNER JOIN InvoiceDetailLastTransformations b ON
        a.Company_DwID = b.Company_DwID AND
        a.BranchDwID = b.BranchInvoice_DwID AND
        a.Product_DwID = b.Product_DwID AND
        a.Serie = b.Serie AND
        a.Form = b.Form AND
        a.Invoice_DwID = b.Invoice_DwID
),
AddColumns AS (
    SELECT 
        a.BranchDwID codigo_agencia,
        a.ChannelDistribution,
        CASE WHEN a.Company_DwID='AR' THEN 'ARGENTINA' ELSE 'REV' END as pais,
        a.cSalesPoint,
        a.Customer_DwID codigo_cliente,
        CONVERT(VARCHAR(10), CAST(a.Emission AS DATE), 120) fecha,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType AG19ID,
        a.Order_DwID,
        a.Product_DwID codigo_producto,
        a.ReasonReturnStock_DwID,
        a.ReasonReturnStock_DwIDDet,
        a.Serie,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') as codigo_ruta,
        a.Valor_Bruto,
        a.Facdsl_ML,
        a.Promocion_ML,
        a.Descuento_ML,
        a.Escalafc_ML,
        a.UnitValue,
        a.Amount,
        CONCAT(
            RIGHT(CONCAT(REPLICATE('0', 4), a.Form), 4), 
            a.Serie, 
            RIGHT(a.cSalesPoint, 4), 
            a.Invoice_DwID
        ) as numero,
        a.ChannelDistribution as cod_linea_ruta,
        'FALSE' as FACCON,
        0 as FACDSL,
        null as dif_precio,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN 0 - facdet
            ELSE facdet
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN a.ReasonReturnStock_DwID
            WHEN NOT (a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX') THEN a.InvoiceCancelReason
            ELSE 'REVISAR'
        END AS Id_Motivo
    FROM FinalJoin a
),
Subchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, a.cIDCustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM cat_rtmgb_dv.groglortmba_cz.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN cat_rtmgb_dv.groglortmba_cz.VW_MC1_CustomerDivisionSector b 
        ON a.cidcustomer = b.cidcustomer
        WHERE b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
JoinWithSubchannel AS (
    SELECT
        a.*,
        b.xSubChannelDistribution
    FROM AddColumns a
    LEFT JOIN  Subchannel b
    ON a.codigo_cliente = b.cIDCustomer
    AND a.codigo_agencia = b.cidbranchinvoice
),
ModifyColumns AS (
    SELECT 
        a.codigo_agencia,
        a.codigo_cliente CLICOD,
        CONVERT(VARCHAR(10), a.fecha, 120) Fecha_vta,
        a.Invoice_DwID no_factura,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.ReasonReturnStock_DwIDDet,
        a.codigo_ruta,
        a.UnitValue precio,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Facdsl_ML
            ELSE a.Facdsl_ML END AS FACDSL$ ,
         CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Promocion_ML
            ELSE a.Promocion_ML END AS promocion,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Descuento_ML
            ELSE a.Descuento_ML END AS descuento,
        a.Amount cantidad,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZBON' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Escalafc_ML
            ELSE a.Escalafc_ML END AS Escalafc_ML,
        a.numero,
        a.dif_precio,
        a.FACCON,
        a.FACDSL,
        a.facdet FACDET,
        CASE  
    		WHEN a.cod_linea_ruta <> a.xSubChannelDistribution AND a.xSubChannelDistribution IS NOT NULL THEN a.xSubChannelDistribution ELSE a.cod_linea_ruta 
		END AS cod_linea_ruta,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            WHEN upper(a.Id_Motivo)<>'WTM001' THEN a.Id_Motivo
            ELSE a.ReasonReturnStock_DwIDDet END as id_motivoventa,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            WHEN upper(a.Id_Motivo)='WTM002' and a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason 
            ELSE a.Id_Motivo END as Id_Motivo
    FROM JoinWithSubchannel a 
),
ReturnReason AS (
    SELECT 
        CASE
            WHEN cDescription IS NOT NULL AND cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_GeneralDescription
    Where cDomainType = 'xReturnReason'
    AND lEnabled = 1
    AND UPPER(cDescription) <> 'SOBRESTOCK EN TIENDA'
),
Motivos AS(
    SELECT 
        CASE
            WHEN cDescription IS NOT NULL AND cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled
    from cat_rtmgb_dv.groglortmba_cz.VW_MC1_GeneralDescription
    Where cDomainType in ('xIDReasonReturnStock', 'xIDMotiveDevolution')
    and cIDLanguage = '2'
    AND lEnabled = 1
    AND UPPER(cDescription) <> 'SOBRESTOCK EN TIENDA'
),
FinalMC1Argentina as (
    SELECT 
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.no_factura,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.FACDSL$,
        a.promocion,
        a.descuento,
        a.cantidad,
        a.Escalafc_ML,
        a.numero,
        a.dif_precio,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.cod_linea_ruta,
        a.id_motivoventa,
        a.Id_Motivo,
        CASE
            WHEN b.Description is null
            THEN b.Description
            ELSE CONCAT(b.Description, '-', c.Description)
        END AS factura_cliente
    FROM ModifyColumns a
    LEFT JOIN ReturnReason b
    ON a.InvoiceCancelReason = b.cIDDomainType
    LEFT JOIN Motivos c
    ON a.id_motivoventa = c.cIDDomainType
    Where a.codigo_ruta NOT LIKE '%0A%'
)
SELECT * FROM FinalMC1Argentina